In [1]:
from langfuse import Langfuse
from anthropic import AnthropicBedrock
from compiler.core import Compiler
from application import Application

In [2]:
langfuse_client = Langfuse()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
m_claude = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [3]:
builder = Application(m_claude, compiler)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [4]:
prepare_res = builder.prepare_bot(["bot that greets me by name"], bot_id="1337_420", capabilities=[])

Processing transition: [] FsmState.TYPESPEC
Processing transition: [<FsmState.TYPESPEC: 'typespec'>] FsmState.WAIT
{'description': '', 'user_requests': ['bot that greets me by name'], 'typespec_schema': <fsm_core.typespec.Success object at 0x7fcc6c714a10>}


In [5]:
prepare_res

ApplicationPrepareOut(refined_description=RefineOut(refined_description='', error_output=None), capabilities=CapabilitiesOut(capabilities=[], error_output=None), typespec=TypespecOut(reasoning='Based on the chat history, I need to create a TypeSpec interface for a greeting bot that greets users by name. The bot should be able to:\n\n1. Greet a user by their name\n2. Store and retrieve user information\n3. Handle different greeting scenarios\n\nLet me define appropriate models and an interface:\n- A model for user information (name)\n- A model for greeting options\n- An interface with functions to greet users\n\nSince we need to adhere to TypeSpec conventions and use the extended decorators, I\'ll make sure each function:\n- Has an @llm_func decorator with a concise description\n- Has at least one @scenario decorator with Gherkin scenarios\n- Takes a single "options" parameter that\'s a model type', typespec_definitions='model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n 

In [6]:
prepare_res.typespec.typespec_definitions

'model UserInfo {\n  name: string;\n}\n\nmodel GreetingOptions {\n  user: UserInfo;\n  timeOfDay?: string;\n  isReturningUser?: boolean;\n}\n\nmodel StoreUserOptions {\n  user: UserInfo;\n}\n\nmodel GetUserOptions {\n  name: string;\n}\n\ninterface GreetingBot {\n  @scenario("""\n  Scenario: Greeting a new user\n  When user provides their name "John"\n  Then system greets "Hello, John! Nice to meet you."\n  """)\n  @scenario("""\n  Scenario: Greeting a user at specific time of day\n  When user provides name "Jane" and time "morning"\n  Then system greets "Good morning, Jane!"\n  """)\n  @scenario("""\n  Scenario: Greeting a returning user\n  When returning user "Alex" connects\n  Then system greets "Welcome back, Alex!"\n  """)\n  @llm_func("Greet a user by their name with appropriate greeting")\n  greetUser(options: GreetingOptions): string;\n\n  @scenario("""\n  Scenario: Storing user information\n  When user information is provided with name "Michael"\n  Then system stores the user 

In [7]:
reasoning = prepare_res.typespec.reasoning
typespec_schema = prepare_res.typespec.typespec_definitions
fake_msg = f"<reasoning>{reasoning}</reasoning><typespec>{typespec_schema}</typespec>"

In [12]:
final_res = builder.update_bot(
    typespec_schema=fake_msg,
    user_requests=["bot that greets user by name", "bot that greets me by surname and age"],
    bot_id="1337_420",
    capabilities=[],
)

Typespec schema: <reasoning>Based on the chat history, I need to create a TypeSpec interface for a greeting bot that greets users by name. The bot should be able to:

1. Greet a user by their name
2. Store and retrieve user information
3. Handle different greeting scenarios

Let me define appropriate models and an interface:
- A model for user information (name)
- A model for greeting options
- An interface with functions to greet users

Since we need to adhere to TypeSpec conventions and use the extended decorators, I'll make sure each function:
- Has an @llm_func decorator with a concise description
- Has at least one @scenario decorator with Gherkin scenarios
- Takes a single "options" parameter that's a model type</reasoning><typespec>model UserInfo {
  name: string;
}

model GreetingOptions {
  user: UserInfo;
  timeOfDay?: string;
  isReturningUser?: boolean;
}

model StoreUserOptions {
  user: UserInfo;
}

model GetUserOptions {
  name: string;
}

interface GreetingBot {
  @scen

In [10]:
final_res

ApplicationOut(refined_description=RefineOut(refined_description='', error_output=None), capabilities=CapabilitiesOut(capabilities=[], error_output=None), typespec=TypespecOut(reasoning="Based on the conversation, the user wants a bot that greets them by surname and age. This would require the bot to:\n1. Store or retrieve the user's surname and age\n2. Generate a greeting using this information\n3. Potentially update this information if it changes\n\nSo I'll create an interface with functions to:\n- Set/update user information (surname and age)\n- Generate a personalized greeting using the stored information\n- Retrieve the stored user information\n\nThe models needed would be:\n- UserInfo: to store surname and age\n- GreetingOptions: for any customization of the greeting\n\nEach function will have appropriate @llm_func descriptions and @scenario decorators to illustrate usage.", typespec_definitions='model UserInfo {\n  surname: string;\n  age: integer;\n}\n\nmodel GreetingOptions {\

In [ ]:
from core.interpolator import Interpolator

import os

# Create the botbuild directory if it doesn't exist
if not os.path.exists("../botbuild"):
    os.makedirs("../botbuild")

interpolator = Interpolator(".")

interpolator.bake(final_res, "../../bot_result")
